In [1]:
from ultralytics.data import YOLODataset
from ultralytics import YOLO
import pandas as pd
# from ultralytics.data.dataset import 

# YOLODataset(
#     data="/home/guycoh/ReliabilityInML_Project/datasets/GlobalWheat/GlobalWheat2020.yaml",
#     img_path="/home/guycoh/ReliabilityInML_Project/datasets/GlobalWheat2020/images")
pd.read_json("runs/detect/val5/predictions.json")

,image_id,category_id,bbox,score
0,001678dc-afb4-4e28-af42-d93c2c91f2f6,0,"[24.0, 37.0, 108.0, 36.0]",1.00000
1,001678dc-afb4-4e28-af42-d93c2c91f2f6,0,"[42.0, 521.0, 74.0, 35.0]",1.00000
2,001678dc-afb4-4e28-af42-d93c2c91f2f6,0,"[790.0, 895.0, 52.0, 70.0]",1.00000
3,001678dc-afb4-4e28-af42-d93c2c91f2f6,0,"[238.999, 973.0, 50.0, 34.0]",1.00000
4,001678dc-afb4-4e28-af42-d93c2c91f2f6,0,"[819.0, 718.0, 64.0, 28.0]",1.00000
...,...,...,...,...
224095,ffe986f0-e22e-4d3e-a34b-e0e240fd0595,0,"[946.207, 651.377, 68.961, 54.417]",0.03586
224096,ffe986f0-e22e-4d3e-a34b-e0e240fd0595,0,"[164.387, 0.0, 100.961, 35.958]",0.03562
224097,ffe986f0-e22e-4d3e-a34b-e0e240fd0595,0,"[415.56, 545.514, 83.055, 37.086]",0.03539
224098,ffe986f0-e22e-4d3e-a34b-e0e240fd0595,0,"[628.969, 551.749, 71.876, 40.896]",0.03461


In [25]:
pd.read_csv("datasets/GlobalWheat2020/labels/arvalis_1/0a4aa505-050f-4a20-82fc-347da4f69c35.txt", 
            sep=" ", 
            names=["class", "x", "y", "w", "h"],
            )

,class,x,y,w,h
0,0,0.666016,0.706055,0.052734,0.070312
1,0,0.676758,0.765625,0.041016,0.083984
2,0,0.695801,0.573242,0.032227,0.035156
3,0,0.695312,0.413574,0.089844,0.053711
4,0,0.274902,0.250977,0.059570,0.039062
...,...,...,...,...,...
70,0,0.714355,0.620605,0.079102,0.071289
71,0,0.565918,0.227539,0.038086,0.039062
72,0,0.044922,0.181152,0.031250,0.045898
73,0,0.371582,0.288086,0.047852,0.027344


In [10]:
model = YOLO(model='ReliabilityInML/wheat16/weights/best.pt', task="detect")
res = model("datasets/GlobalWheat2020/images/arvalis_3/0a82cf5a-83df-453e-bc85-2852cce9ddc2.png")
# model.val(
#     # save_dir='',
#     imgsz=1024,
#     save_json=True,
#     save_hybrid=True,
#     split='test',
#     #   conf=0.1,
#     #  iou=0.6,
#     )


image 1/1 /home/guycoh/ReliabilityInML_Project/datasets/GlobalWheat2020/images/arvalis_3/0a82cf5a-83df-453e-bc85-2852cce9ddc2.png: 1024x1024 2 wheat_heads, 97.5ms
Speed: 3.3ms preprocess, 97.5ms inference, 555.3ms postprocess per image at shape (1, 3, 1024, 1024)


In [20]:
df = res[0].to_df()
df['box'][0]

{'x1': 690.22729, 'y1': 907.04358, 'x2': 835.33081, 'y2': 990.67712}

In [2]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
from wilds.datasets.globalwheat_dataset import GlobalWheatDataset

from neural.config import TrainConfig

In [1]:
import torch
torch.__version__

'2.4.1'

In [ ]:
settings = TrainConfig(
        # loss_fn = 'bce', # str
        # optimizer = 'adam', # str
        device = f"cpu", # str
        log = True, # bool
        logs_dir = "logs/", # str
        num_epochs = 30, # int
        checkpoints = 'test.pt', # str
        early_stopping = None, # int
        log_every = 1, # int
        timeout = "00:12:00:00", # int
        # learning_rate = 0.001, # float
        # weight_decay = 1e-06, # float
        batch_size = 50, # int
        shuffle = True, # bool
        num_workers = 9, # int
        train_test_split = 0.5, # float
    )

In [3]:

# Load the full dataset, and download it if necessary
dataset:GlobalWheatDataset = get_dataset(dataset="globalwheat", download=True)


In [4]:
print(dataset.metadata_fields)
print(dataset.DEFAULT_SPLIT_NAMES)

['session', 'country', 'location', 'stage', 'from_source_domain']
{'train': 'Train', 'val': 'Validation', 'test': 'Test'}


In [8]:
# Get the training set
train_data = dataset.get_subset("train")
val_data = dataset.get_subset('val')

In [7]:
train_data.indices

array([   0,    1,    2, ..., 2940, 2941, 2942])

In [9]:
val_data.indices

array([2943, 2944, 2945, ..., 4364, 4365, 4366])

In [ ]:
# Prepare the standard data loader
train_loader = get_train_loader("standard", train_data, batch_size=16)

In [ ]:
# unlabeled_data = dataset.get_subset(
#     "test_unlabeled",
#     transform=transforms.Compose(
#         [transforms.Resize((448, 448)), transforms.ToTensor()]
#     ),
# )
# unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)

In [15]:
from wilds.common.grouper import CombinatorialGrouper

# Initialize grouper, which extracts domain information
# In this example, we form domains based on location
grouper = CombinatorialGrouper(dataset, ['location'])


tensor([1])

In [ ]:
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...